## 新能源电动汽车动力电池充电能量预测（五）——模型训练

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from datetime import date, datetime

### 导入数据

In [5]:
# 导入经过填充之后的数据集合
dataset = pd.read_csv('./energy_predict_data/train_data_filter/4_fill_null/whole/filled_by_kmeans/predict_data_e_train.csv')
# 数据字段
header = dataset.columns.values.tolist()
# 数据集大小
count = np.max(dataset['vehicle_id'])
# 数据描述
pd.set_option('precision', 2)
dataset.describe()

,vehicle_id,charge_start_time,charge_end_time,charge_duration,mileage,delta_mileage,charge_start_soc,charge_end_soc,charge_delta_soc,charge_start_U,charge_end_U,charge_start_I,charge_end_I,charge_max_temp,charge_min_temp,charge_energy
count,4818.00,4.82e+03,4.82e+03,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00,4818.00
mean,5.43,2.02e+13,2.02e+13,4346.83,111192.41,98.94,38.10,64.27,26.17,440.65,415.62,-54.07,-23.52,28.23,12.70,28.46
std,2.97,4.04e+09,4.04e+09,4628.47,32314.21,219.41,20.73,34.70,33.31,102.62,162.30,57.55,43.73,10.99,12.03,24.10
min,1.00,2.02e+13,2.02e+13,10.00,45631.70,-146.40,0.80,0.00,-100.00,252.00,0.00,-202.00,-400.00,-2.00,-7.00,0.10
25%,3.00,2.02e+13,2.02e+13,1630.00,84895.15,61.10,21.60,32.80,11.60,339.20,370.50,-84.50,-16.20,19.00,0.00,11.51
50%,5.00,2.02e+13,2.02e+13,3085.50,109703.30,86.00,36.00,79.00,26.40,371.90,375.60,-24.80,-8.60,30.00,9.00,19.87
75%,8.00,2.02e+13,2.02e+13,4920.00,138055.25,122.98,53.00,100.00,51.00,540.77,565.08,-12.03,-2.00,36.00,23.00,41.42
max,10.00,2.02e+13,2.02e+13,42770.00,184014.00,9703.00,100.00,100.00,88.00,585.10,604.80,-0.10,9.00,52.00,44.00,109.84


### 选取不同的车辆 

### 特征选取

In [46]:
features = ['charge_duration',
 'delta_mileage',
 'mileage',
 'charge_start_soc',
 'charge_delta_soc',
 'charge_end_soc',
 'charge_start_U',
 'charge_end_U',
 'charge_start_I',
 'charge_end_I',
 'charge_max_temp',
 'charge_min_temp']
label = header[15]
label

'charge_energy'

In [10]:
X = dataset[features]
y = dataset[label]

### 创建训练后的预测值数据集

In [6]:
data_proced = pd.DataFrame()

In [7]:
data_proced['id'] = dataset['id']

In [8]:
data_proced['type'] = 3 #补充空值

### 划分训练集和测试集

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)##训练集划分

### 标准化

In [12]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
X_train = scaler.fit_transform(X_train.astype(np.float64))
X_test = scaler.fit_transform(X_test.astype(np.float64))
X = scaler.fit_transform(X.astype(np.float64))

In [13]:
### 开始算法

In [14]:
#导入sklearn库中的VarianceThreshold
from sklearn.feature_selection import VarianceThreshold
#设置方差的阈值为0.08
sel = VarianceThreshold(threshold=0.08)
#选择方差大于0.08的特征
X_sel = sel.fit_transform(X_train.astype(np.float64))

In [15]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
y_pred = linreg.predict(X)

In [17]:
#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y, y_pred))
# 计算精确度
# for i in range(len(y_pred)):
#     if y_pred[i] < 0:
#         y_pred[i] = 0.4
accuracy = np.mean(abs(y - y_pred) / y)
print("accuracy: %f" % accuracy)

MSE: 78.8939868239
RMSE: 8.88222870815
MAE: 6.50559411867
accuracy: 1.568705


In [18]:
data_proced['lr'] = y_pred

NameError: name 'data_proced' is not defined

In [19]:
from sklearn.preprocessing import PolynomialFeatures
featurizer = PolynomialFeatures(degree=3)
X_train_cubic = featurizer.fit_transform(X_train)
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train_cubic, y_train)
X_test_cubic = featurizer.transform(X_test)
y_pred = linreg.predict(X_test_cubic)

In [20]:
#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))
# 计算精确度
# for i in range(len(y_pred)):
#     if y_pred[i] < 0:
#         y_pred[i] = 0.04

error_array = np.array(((y_test - y_pred) / y_test))
error_rate = np.sqrt(np.sum(np.power(error_array,2)))
print("error_rate: %f" % error_rate)

MSE: 4.76619071556e+18
RMSE: 2183160716.84
MAE: 1867551879.43
error_rate: 67130783904.904472


In [21]:
X_te11 = featurizer.fit_transform(X)
y_pred = linreg.predict(X_te11)

In [22]:
data_proced['plr'] = y_pred

NameError: name 'data_proced' is not defined

In [23]:
from sklearn.svm import SVR
l_svr = SVR(kernel='linear')
l_svr.fit(X_train,y_train)
y_pred = l_svr.predict(X_test)
#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

MSE: 90.0217438813
RMSE: 9.48797891446
MAE: 6.49898778776


In [24]:
y_pred = l_svr.predict(X)

In [23]:
data_proced['svr_lr'] = y_pred

###  svr poly不进行标准化死循环

import time
from sklearn.svm import SVR
l_svr=SVR(kernel='poly')
#记录训练开始时间
time_start=time.clock()
l_svr.fit(X_train,y_train)
#记录训练结束时间
time_end=time.clock()
print('Running time: %s Seconds'%(time_end-time_start))
l_svr.score(X_test,y_test)

In [25]:
from sklearn.svm import SVR
n_svr = SVR(kernel="poly")
n_svr.fit(X_train,y_train)
y_pred = n_svr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

MSE: 260.297400263
RMSE: 16.1337348516
MAE: 10.9611986888


In [26]:
y_pred = n_svr.predict(X)

In [26]:
data_proced['svr_plr'] = y_pred

###  继续其他算法

In [27]:
import time
from sklearn.svm import SVR
l_svr = SVR(kernel='rbf')
#记录训练开始时间
time_start=time.clock()
l_svr.fit(X_train,y_train)
#记录训练结束时间
time_end=time.clock()
print('Running time: %s Seconds'%(time_end-time_start))
l_svr.score(X_test,y_test)

Running time: 0.543704 Seconds


0.78372541123921002

In [28]:
y_pred=l_svr.predict(X)

In [29]:
data_proced['svr_rbf'] = y_pred

###  dtr 回归树

In [29]:
import time
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
#记录训练开始时间
time_start = time.clock()
dt.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = dt.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.03218800000000055 Seconds
MSE: 89.1769597585
RMSE: 9.44335532311
MAE: 5.26445892116


In [30]:
y_pred = dt.predict(X)
data_proced['dtr'] = y_pred

NameError: name 'data_proced' is not defined

### etr极端森林

In [31]:
import time
from sklearn.ensemble import ExtraTreesRegressor
etr = ExtraTreesRegressor()
#记录训练开始时间
time_start = time.clock()
etr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = etr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.1122999999999994 Seconds
MSE: 29.3130967435
RMSE: 5.41415706675
MAE: 3.37878999664


In [32]:
y_pred = etr.predict(X)
data_proced['etr'] = y_pred

NameError: name 'data_proced' is not defined

###  rtr随机森林

In [33]:
import time
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
#记录训练开始时间
time_start = time.clock()
rfr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = rfr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.19997500000000024 Seconds
MSE: 53.3051808722
RMSE: 7.30103971173
MAE: 4.33296542055


In [34]:
y_pred = rfr.predict(X)
data_proced['rtr'] = y_pred

NameError: name 'data_proced' is not defined

### 自适应增强决策树模型预测值

In [35]:
from sklearn.ensemble import AdaBoostRegressor
abr = AdaBoostRegressor()
time_start = time.clock()
abr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = abr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.2945330000000004 Seconds
MSE: 63.051968692
RMSE: 7.94052697823
MAE: 6.39324150035


In [36]:
y_pred = abr.predict(X)
data_proced['abr'] = y_pred

NameError: name 'data_proced' is not defined

### br BaggingRegressor模型预测值

In [37]:
from sklearn.ensemble import BaggingRegressor
br = BaggingRegressor()
time_start = time.clock()
br.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = br.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.2125679999999992 Seconds
MSE: 48.9162487532
RMSE: 6.99401520967
MAE: 4.31823482646


In [38]:
y_pred = br.predict(X)
data_proced['br'] = y_pred

NameError: name 'data_proced' is not defined

### GradientBoostingRegressor

In [39]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()
time_start = time.clock()
gbr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = gbr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.2511939999999999 Seconds
MSE: 72.504150398
RMSE: 8.51493689924
MAE: 5.64233772299


In [40]:
y_pred = gbr.predict(X)
data_proced['gbr'] = y_pred

NameError: name 'data_proced' is not defined

###  XGBRegressor

In [41]:
from xgboost import XGBRegressor
xgbr = GradientBoostingRegressor()
time_start = time.clock()
xgbr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = xgbr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

/Users/Jeremy/miniconda3/envs/test/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Running time: 0.26371500000000037 Seconds
MSE: 72.7972775048
RMSE: 8.53213206091
MAE: 5.64291533819


In [42]:
y_pred = xgbr.predict(X)
data_proced['xgbr'] = y_pred

NameError: name 'data_proced' is not defined

### import MLPRegressor

In [43]:
from sklearn.neural_network import MLPRegressor
mlpr = GradientBoostingRegressor()
time_start = time.clock()
mlpr.fit(X_train,y_train)
#记录训练结束时间
time_end = time.clock()
print('Running time: %s Seconds'%(time_end -time_start))
y_pred = mlpr.predict(X_test)

#评估模型
from sklearn import metrics
# 用scikit-learn计算MSE
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
# 用scikit-learn计算RMSE
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
# 用scikit-learn计算MAE
print("MAE:",metrics.mean_absolute_error(y_test, y_pred))

Running time: 0.24348599999999898 Seconds
MSE: 71.7820191896
RMSE: 8.47242699523
MAE: 5.60498029694


In [44]:
y_pred = mlpr.predict(X)
data_proced['mlpr'] = y_pred

NameError: name 'data_proced' is not defined

### bp神经网络 不进行标准化预测值为NAN

In [45]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import regularizers
model = Sequential()  #层次模型
model.add(Dense(24,input_dim=12,init='uniform'))
model.add(Activation('relu'))  #添加激活函数
model.add(Dense(1,input_dim=24))  #输出层

# 自定义loss函数
from keras import backend as K
def my_loss(y_true,y_pred):
    return K.sqrt(K.sum(K.square((y_pred - y_true) / y_true),axis = -1))
model.compile(loss='mean_squared_error', optimizer='sgd') #编译模型
model.fit(X_train, y_train, nb_epoch = 1000, batch_size = 20) #训练模型1000次

Using TensorFlow backend.
/Users/Jeremy/miniconda3/envs/test/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/Jeremy/miniconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(24, input_dim=12, kernel_initializer="uniform")`
  """
/Users/Jeremy/miniconda3/envs/test/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1000
3613/3613 [==============================] - 0s 87us/step - loss: 1393.9199
Epoch 2/1000
3613/3613 [==============================] - 0s 65us/step - loss: 587.6135
Epoch 3/1000
3613/3613 [==============================] - 0s 67us/step - loss: 534.5454
Epoch 4/1000
3613/3613 [==============================] - 0s 58us/step - loss: 300.3561
Epoch 5/1000
3613/3613 [==============================] - 0s 55us/step - loss: 289.6848
Epoch 6/1000
3613/3613 [==============================] - 0s 54us/step - loss: 639.4294
Epoch 7/1000
3613/3613 [==============================] - 0s 80us/step - loss: 587.8684
Epoch 8/1000
3613/3613 [==============================] - 0s 59us/step - loss: 587.7818
Epoch 9/1000
3613/3613 [==============================] - 0s 52us/step - loss: 587.7702
Epoch 10/1000
3613/3613 [==============================] - 0s 63us/step - loss: 587.8100
Epoch 11/1000
3613/3613 [==============================] - 0s 61us/step - loss: 587.8367
Epoch 12/1000
3613/3613 [====

3613/3613 [==============================] - 0s 73us/step - loss: 587.7833
Epoch 94/1000
3613/3613 [==============================] - 0s 75us/step - loss: 587.7365
Epoch 95/1000
3613/3613 [==============================] - 0s 82us/step - loss: 587.7682
Epoch 96/1000
3613/3613 [==============================] - 0s 63us/step - loss: 587.7105
Epoch 97/1000
3613/3613 [==============================] - 0s 100us/step - loss: 587.0718
Epoch 98/1000
3613/3613 [==============================] - 0s 88us/step - loss: 587.7873: 0s - loss: 598.
Epoch 99/1000
3613/3613 [==============================] - 0s 116us/step - loss: 587.5229
Epoch 100/1000
3613/3613 [==============================] - 0s 114us/step - loss: 587.4083
Epoch 101/1000
3613/3613 [==============================] - 0s 92us/step - loss: 587.7388
Epoch 102/1000
3613/3613 [==============================] - 0s 105us/step - loss: 587.8153
Epoch 103/1000
3613/3613 [==============================] - 0s 98us/step - loss: 587.5487
Epoch 104/

3613/3613 [==============================] - 1s 159us/step - loss: 587.7301
Epoch 184/1000
3613/3613 [==============================] - 1s 213us/step - loss: 587.7364
Epoch 185/1000
3613/3613 [==============================] - 1s 203us/step - loss: 587.6767
Epoch 186/1000
3613/3613 [==============================] - 1s 218us/step - loss: 587.7543
Epoch 187/1000
3613/3613 [==============================] - 1s 145us/step - loss: 587.7257
Epoch 188/1000
3613/3613 [==============================] - 0s 113us/step - loss: 587.6990
Epoch 189/1000
3613/3613 [==============================] - 0s 61us/step - loss: 587.8034
Epoch 190/1000
3613/3613 [==============================] - 0s 58us/step - loss: 587.6082
Epoch 191/1000
3613/3613 [==============================] - 0s 53us/step - loss: 587.8751
Epoch 192/1000
3613/3613 [==============================] - 0s 51us/step - loss: 587.4167
Epoch 193/1000
3613/3613 [==============================] - 0s 52us/step - loss: 587.6098
Epoch 194/1000
3613

3613/3613 [==============================] - 0s 54us/step - loss: 587.7684
Epoch 274/1000
3613/3613 [==============================] - 0s 54us/step - loss: 587.6508
Epoch 275/1000
3613/3613 [==============================] - 0s 52us/step - loss: 587.8069
Epoch 276/1000
3613/3613 [==============================] - 0s 53us/step - loss: 587.8299
Epoch 277/1000
3613/3613 [==============================] - 0s 51us/step - loss: 587.6164
Epoch 278/1000
3613/3613 [==============================] - 0s 53us/step - loss: 587.7334
Epoch 279/1000
3613/3613 [==============================] - 0s 50us/step - loss: 587.6502
Epoch 280/1000
3613/3613 [==============================] - 0s 73us/step - loss: 587.6324
Epoch 281/1000
3613/3613 [==============================] - 0s 109us/step - loss: 587.6824
Epoch 282/1000
3613/3613 [==============================] - 0s 108us/step - loss: 587.8319
Epoch 283/1000
3613/3613 [==============================] - 0s 94us/step - loss: 587.6872
Epoch 284/1000
3613/361

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X)
data_proced['bpr'] = y_pred

In [ ]:
data_proced.to_csv('data_pre_3_%d.csv' % i, index = False)